## Notebook de projeção das querys

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Inicializar a sessão Spark
spark = SparkSession.builder.appName("Analise de Empréstimos").getOrCreate()

# Carregar os dados para DataFrames Spark (exemplo de caminho, ajuste conforme necessário)
df_loans = spark.read.format("csv").option("header", "true").load("caminho_para_loans.csv")
df_clients = spark.read.format("csv").option("header", "true").load("caminho_para_clients.csv")

# Registrando DataFrames como tabelas temporárias para usar com Spark SQL
df_loans.createOrReplaceTempView("loans")
df_clients.createOrReplaceTempView("clients")

### Query 1: Total de Empréstimos e Montante por Mês
Ordendando por total_loans e total_amount

In [ ]:
query1_1 = """
SELECT
    DATE_FORMAT(to_date(created_at), 'yyyy-MM') AS month_year,
    COUNT(loan_id) AS total_loans,
    SUM(loan_amount) AS total_amount
FROM
    loans
GROUP BY
    month_year
ORDER BY
    total_loans DESC
LIMIT
    1
"""
result1_1 = spark.sql(query1_1)
result1_1.display()

In [ ]:
query1_2 = """
SELECT
    DATE_TRUNC('month', created_at) AS month_year,
    COUNT(loan_id) AS total_loans,
    SUM(loan_amount) AS total_amount
FROM
    loans
GROUP BY
    month_year
ORDER BY
    total_amount DESC
LIMIT
    1;
"""
result1_2 = spark.sql(query1_2)
result1_2.display()

### Query 2: Análise de Aderência dos Clientes aos Empréstimos por Batch

Esta análise tem como objetivo calcular a aderência dos clientes aos empréstimos por batch. A aderência é definida como a proporção de clientes em um determinado batch que possuem empréstimos, o que pode indicar a aceitação das condições de empréstimo ou a eficácia das estratégias de marketing direcionadas a esses grupos. Batches com alta aderência podem sugerir uma boa aceitação, enquanto batches com baixa aderência podem necessitar de investigação adicional.


##### Preparação da Query

A query é estruturada em três partes principais usando Common Table Expressions (CTEs):

1. `loan_counts`: Calcula o número total de clientes únicos com empréstimos em cada batch.
2. `total_clients`: Determina o número total de clientes em cada batch.
3. `adherence`: Combina os resultados das CTEs anteriores para calcular a taxa de aderência de cada batch.


In [ ]:
query2 = """
WITH loan_counts AS (
    SELECT
        c.batch,
        COUNT(DISTINCT l.user_id) AS total_clients_with_loans
    FROM
        loans l
    JOIN
        clients c ON l.user_id = c.user_id
    GROUP BY
        c.batch
),
total_clients AS (
    SELECT
        batch,
        COUNT(user_id) AS total_clients_in_batch
    FROM
        clients
    GROUP BY
        batch
),
adherence AS (
    SELECT
        lc.batch,
        tc.total_clients_in_batch,
        lc.total_clients_with_loans,
        (CAST(lc.total_clients_with_loans AS FLOAT) / tc.total_clients_in_batch) AS adherence
    FROM
        loan_counts lc
    JOIN
        total_clients tc ON lc.batch = tc.batch
)
SELECT
    batch,
    total_clients_in_batch,
    total_clients_with_loans,
    adherence
FROM
    adherence
ORDER BY
    adherence DESC
"""
result2 = spark.sql(query2)
result2.display()

### Query 3: Análise da Relação entre Taxa de Juros e Inadimplência
Esta análise visa compreender como as diferentes taxas de juros aplicadas aos clientes afetam a probabilidade de inadimplência nos empréstimos. O foco é identificar se existe uma correlação entre taxas de juros mais altas e uma maior taxa de inadimplência, sugerindo que empréstimos com juros mais elevados podem representar um risco maior para a instituição financeira. Alternativamente, examina-se se taxas de juros mais baixas resultam em menores taxas de inadimplência, considerando também outros fatores como o perfil de crédito do cliente e o prazo do empréstimo.


##### Preparação da Query

A query agrupa os empréstimos pela taxa de juros aplicada, contabiliza o total de empréstimos e calcula tanto o número total de empréstimos inadimplentes quanto a taxa de inadimplência para cada taxa de juros. A ordenação final por taxa de juros permite uma análise ascendente que facilita a visualização da correlação entre taxa de juros e inadimplência.

In [ ]:
query3 = """
SELECT
    c.interest_rate,
    COUNT(l.loan_id) AS total_loans,
    SUM(CASE WHEN l.status = 'default' THEN 1 ELSE 0 END) AS defaults,
    SUM(CASE WHEN l.status = 'default' THEN 1 ELSE 0 END)::FLOAT / COUNT(l.loan_id) AS default_rate
FROM
    loans l
JOIN
    clients c ON l.user_id = c.user_id
GROUP BY
    c.interest_rate
ORDER BY
    c.interest_rate
"""

result3 = spark.sql(query3)
result3.display()


### Querys 4.0, 4.1 e 4.2: Classificação de Clientes Baseada em Confiabilidade de Pagamento
Esta análise foca em classificar os clientes com base em sua confiabilidade de pagamento, a presença de inadimplência e o volume total de empréstimos. O objetivo é identificar os clientes com melhor e pior desempenho, segmentando-os como "Top Clients" e "Bottom Clients". Esta classificação ajuda a instituição financeira a entender melhor o comportamento de pagamento dos seus clientes e a identificar áreas potenciais de risco e oportunidade.


##### Estrutura da Query 4.0

A query é dividida em várias partes, utilizando Common Table Expressions (CTEs) para uma análise passo a passo:

1. **`client_scores`:** Calcula métricas chave para cada cliente, incluindo a confiabilidade média de pagamento, se houve algum empréstimo em default, e o valor total dos empréstimos.
2. **`ranked_clients`:** Utiliza os dados do CTE `client_scores` para classificar os clientes, atribuindo rankings baseados em confiabilidade de pagamento, presença de inadimplência e volume total de empréstimos.
3. **`top_clients` e `bottom_clients`:** Segmenta os clientes nos 10 melhores e piores com base nos rankings calculados anteriormente.
4. **Seleção Final:** Combina os segmentos de "Top Clients" e "Bottom Clients" para uma visualização consolidada, ordenando os resultados para destacar as diferenças em desempenho.


In [ ]:
query4_0 = """
WITH client_scores AS (
    SELECT
        l.user_id,
        AVG(l.amount_paid / l.due_amount) AS avg_payment_reliability,
        BOOL_OR(l.status = 'default') AS any_default,
        SUM(l.loan_amount) AS total_loan_amount
    FROM
        loans l
    GROUP BY
        l.user_id
),
ranked_clients AS (
    SELECT
        cs.user_id,
        cs.avg_payment_reliability,
        cs.any_default,
        cs.total_loan_amount,
        ROW_NUMBER() OVER (ORDER BY cs.avg_payment_reliability DESC, NOT cs.any_default DESC, cs.total_loan_amount DESC) AS rank_asc,
        ROW_NUMBER() OVER (ORDER BY cs.avg_payment_reliability, cs.any_default, cs.total_loan_amount) AS rank_desc
    FROM
        client_scores cs
),
top_clients AS (
    SELECT
        'Top Clients' AS category,
        user_id,
        avg_payment_reliability,
        any_default,
        total_loan_amount
    FROM
        ranked_clients
    WHERE
        rank_asc <= 10
),
bottom_clients AS (
    SELECT
        'Bottom Clients' AS category,
        user_id,
        avg_payment_reliability,
        any_default,
        total_loan_amount
    FROM
        ranked_clients
    WHERE
        rank_desc <= 10
)
SELECT * FROM top_clients
UNION ALL
SELECT * FROM bottom_clients
ORDER BY category DESC, avg_payment_reliability DESC, any_default, total_loan_amount DESC
"""

result4_0 = spark.sql(query4_0)
result4_0.display()

##### Estrutura da Query 4.1

A query é organizada em várias etapas utilizando Common Table Expressions (CTEs):

1. **`client_details`:** Agrupa os dados de empréstimos por cliente, calculando métricas como a confiabilidade média de pagamento, presença de inadimplência, classificação do perfil de empréstimo baseado no valor do empréstimo, e outros indicadores relevantes.
2. **`client_scores`:** Utiliza as métricas calculadas para atribuir um score a cada cliente, considerando fatores como a ausência de inadimplência, confiabilidade de pagamento, se é um cliente recorrente, entre outros.
3. **`ranked_clients`:** Classifica os clientes com base nos scores calculados, identificando os 10 melhores e piores clientes.
4. **Seleção Final:** Filtra os melhores e piores clientes e os categoriza, ordenando os resultados para facilitar a análise.


In [ ]:
query4_1 = """
WITH client_details AS (
    SELECT
        l.user_id,
        BOOL_OR(l.status = 'default') AS any_default,
        AVG(l.amount_paid / l.due_amount) AS avg_payment_reliability,
        COUNT(l.loan_id) > 1 AS is_recurrent_client,
        BOOL_OR(l.loan_amount > 20000) AS is_profile_a,
        BOOL_OR(l.loan_amount BETWEEN 5000 AND 20000) AS is_profile_b,
        BOOL_OR(l.loan_amount < 5000) AS is_profile_c,
        SUM(l.loan_amount) AS total_loan_amount,
        SUM(l.due_amount - l.amount_paid) AS total_unpaid,
        BOOL_OR(l.amount_paid < l.due_amount AND l.paid_at IS NOT NULL) AS has_partial_payments
    FROM
        loans l
    GROUP BY
        l.user_id
),
client_scores AS (
    SELECT
        cd.user_id,
        cd.any_default,
        cd.avg_payment_reliability,
        cd.is_recurrent_client,
        cd.is_profile_a,
        cd.is_profile_b,
        cd.is_profile_c,
        cd.total_loan_amount,
        cd.total_unpaid,
        cd.has_partial_payments,
        CASE 
            WHEN cd.is_profile_a THEN 'A'
            WHEN cd.is_profile_b THEN 'B'
            WHEN cd.is_profile_c THEN 'C'
            ELSE 'Unknown'
        END AS loan_profile,
        (
            (CASE WHEN NOT cd.any_default THEN 0.4 ELSE 0 END) + 
            (CASE WHEN cd.avg_payment_reliability >= 0.95 THEN 0.3 ELSE 0 END) + 
            (CASE WHEN cd.is_recurrent_client THEN 0.2 ELSE 0 END) + 
            (CASE WHEN cd.has_partial_payments THEN -0.1 ELSE 0.1 END)
        ) - 
        (CASE WHEN cd.total_unpaid > 0 THEN 0.2 ELSE 0 END) AS score
    FROM
        client_details cd
),
ranked_clients AS (
    SELECT
        cs.*,
        ROW_NUMBER() OVER (ORDER BY cs.score DESC) AS rank_asc,
        ROW_NUMBER() OVER (ORDER BY cs.score) AS rank_desc
    FROM
        client_scores cs
)
SELECT
    user_id,
    loan_profile,
    avg_payment_reliability,
    any_default,
    is_recurrent_client,
    has_partial_payments,
    total_loan_amount,
    score,
    CASE
        WHEN rank_asc <= 10 THEN 'Top Clients'
        WHEN rank_desc <= 10 THEN 'Bottom Clients'
        ELSE 'Middle Clients'
    END AS category
FROM
    ranked_clients
WHERE
    rank_asc <= 10 OR rank_desc <= 10
ORDER BY
    category DESC, rank_asc
"""

result4_1 = spark.sql(query4_1)
result4_1.display()

##### Estrutura da Query 4.2

A query é organizada nas seguintes etapas, utilizando Common Table Expressions (CTEs) para facilitar a análise:

1. **`client_details`:** Calcula métricas-chave para cada cliente, incluindo a presença de inadimplência, a confiabilidade média de pagamento, se o cliente é recorrente, entre outros.
2. **`client_scores`:** Atribui um score a cada cliente com base nas métricas calculadas, classificando os clientes em perfis de empréstimo e ajustando a pontuação para refletir a ausência de inadimplência, confiabilidade de pagamento, etc.
3. **`ranked_clients`:** Classifica os clientes com base nos scores calculados, determinando os 10 melhores e piores clientes.
4. **Seleção Final:** Filtra e categoriza os melhores e piores clientes, ordenando os resultados para destacar as diferenças em desempenho.

Análise Multifacetada do Comportamento dos Clientes em Relação aos Empréstimos

Esta análise detalhada foca em avaliar o comportamento dos clientes baseando-se em vários critérios relacionados aos seus empréstimos. Utilizando uma metodologia de pontuação customizada, identificamos os clientes com melhor e pior desempenho financeiro. Este método considera fatores como inadimplência, confiabilidade de pagamento, recorrência de empréstimos e comportamentos de pagamento parcial, proporcionando uma visão abrangente da base de clientes.

In [ ]:
query4_2 = """
WITH client_details AS (
    SELECT
        l.user_id,
        BOOL_OR(l.status = 'default') AS any_default,
        AVG(l.amount_paid / l.due_amount) AS avg_payment_reliability,
        COUNT(l.loan_id) > 1 AS is_recurrent_client,
        BOOL_OR(l.loan_amount > 20000) AS is_profile_a,
        BOOL_OR(l.loan_amount BETWEEN 5000 AND 20000) AS is_profile_b,
        BOOL_OR(l.loan_amount < 5000) AS is_profile_c,
        SUM(l.loan_amount) AS total_loan_amount,
        SUM(l.due_amount - l.amount_paid) AS total_unpaid,
        BOOL_OR(l.amount_paid < l.due_amount AND l.paid_at IS NOT NULL) AS has_partial_payments
    FROM
        loans l
    GROUP BY
        l.user_id
),
client_scores AS (
    SELECT
        cd.user_id,
        cd.any_default,
        cd.avg_payment_reliability,
        cd.is_recurrent_client,
        cd.total_loan_amount,
        cd.total_unpaid,
        cd.has_partial_payments,
        CASE 
            WHEN cd.is_profile_a THEN 'A'
            WHEN cd.is_profile_b THEN 'B'
            WHEN cd.is_profile_c THEN 'C'
            ELSE 'Unknown'
        END AS loan_profile,
        (
            (CASE WHEN NOT cd.any_default THEN 0.4 ELSE 0 END) +
            (CASE WHEN cd.avg_payment_reliability >= 0.95 THEN 0.3 ELSE 0 END) +
            (CASE WHEN cd.is_recurrent_client THEN 0.2 ELSE 0 END) +
            (CASE WHEN cd.has_partial_payments THEN -0.1 ELSE 0.1 END)
        ) - 
        (CASE WHEN cd.total_unpaid > 0 THEN 0.2 ELSE 0 END) AS score
    FROM
        client_details cd
),
ranked_clients AS (
    SELECT
        cs.*,
        ROW_NUMBER() OVER (ORDER BY cs.score DESC) AS rank_asc,
        ROW_NUMBER() OVER (ORDER BY cs.score ASC) AS rank_desc
    FROM
        client_scores cs
)
SELECT
    user_id,
    loan_profile,
    avg_payment_reliability,
    any_default,
    is_recurrent_client,
    has_partial_payments,
    total_loan_amount,
    score,
    CASE
        WHEN rank_asc <= 10 THEN 'Top Clients'
        WHEN rank_desc <= 10 THEN 'Bottom Clients'
        ELSE 'Middle Clients'
    END AS category
FROM
    ranked_clients
WHERE
    rank_asc <= 10 OR rank_desc <= 10
ORDER BY
    score DESC, rank_asc
"""

result4_2 = spark.sql(query4_2)
result4_2.display()

### Query 5: Análise de Performance dos Empréstimos por Mês e Batch

Esta consulta SQL é projetada para fornecer insights sobre a performance dos empréstimos ao longo do tempo, agrupando os dados por mês e batch. O foco está em avaliar o volume de empréstimos emitidos, o número de casos de inadimplência, e a taxa geral de inadimplência, o que pode ajudar a instituição financeira a identificar padrões, avaliar a eficácia das políticas de crédito e ajustar estratégias conforme necessário.

##### Estrutura da Query 5

A query realiza as seguintes operações:

1. **Seleção de Dados:** Extrai a data de criação do empréstimo, o batch do cliente, e calcula o total de empréstimos e inadimplentes.
2. **Agrupamento:** Organiza os dados por mês e batch, permitindo uma análise temporal e por segmento de clientes.
3. **Cálculo da Taxa de Inadimplência:** Determina a proporção de empréstimos inadimplentes em relação ao total de empréstimos para cada grupo.
4. **Ordenação:** Arranja os resultados primeiro por mês de forma ascendente para uma análise cronológica, e dentro de cada mês, destaca os batches com maior taxa de inadimplência.


In [ ]:
query5 = """
SELECT
    DATE_FORMAT(to_date(l.created_at), 'yyyy-MM') AS month_year,
    c.batch,
    COUNT(l.loan_id) AS total_emprestimos,
    SUM(CASE WHEN l.status = 'default' THEN 1 ELSE 0 END) AS inadimplentes,
    SUM(CASE WHEN l.status = 'default' THEN 1 ELSE 0 END) / COUNT(l.loan_id) AS taxa_inadimplencia
FROM
    loans l
JOIN
    clients c ON l.user_id = c.user_id
GROUP BY
    month_year, c.batch
ORDER BY
    month_year ASC,
    taxa_inadimplencia DESC,
    c.batch ASC
"""
result5 = spark.sql(query5)
result5.display()

### Querys 6: Análise Financeira dos Empréstimos

Esta consulta SQL oferece uma visão geral financeira dos empréstimos, calculando o total emprestado, o total recebido dos clientes, o total de inadimplência, e o lucro operacional resultante. Essas métricas fornecem insights cruciais sobre a saúde financeira da operação de empréstimos, ajudando a instituição a avaliar a eficácia das suas políticas de crédito e estratégias de cobrança.

##### Estrutura da Query 6

A query executa as seguintes operações:

1. **Total Emprestado (`total_emprestado`):** Soma de todos os valores de empréstimos emitidos.
2. **Total Recebido (`total_recebido`):** Soma de todos os pagamentos recebidos em relação aos empréstimos.
3. **Total de Inadimplência (`total_inadimplencia`):** Soma dos montantes devidos apenas dos empréstimos que estão em default.
4. **Lucro Operacional:** Calculado como a diferença entre o total recebido e o total de inadimplência, representando o lucro gerado pela atividade de empréstimo após considerar as perdas por inadimplência.


In [ ]:
query6 = """
SELECT
    SUM(l.loan_amount) AS total_emprestado,
    SUM(l.amount_paid) AS total_recebido,
    SUM(CASE WHEN l.status = 'default' THEN l.due_amount ELSE 0 END) AS total_inadimplencia,
    (SUM(l.amount_paid) - SUM(CASE WHEN l.status = 'default' THEN l.due_amount ELSE 0 END)) AS lucro_operacional
FROM
    loans l
"""
result6 = spark.sql(query6)
result6.display()